In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebrasss
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torchsummary

# 1. 데이터 전처리

In [ ]:
import torch # 파이토치 기본 라이브러리 
import torchvision # 이미지 관련 된 파이토치 라이브러리
from torchvision import datasets # 토치비전에서 제공하는 데이터셋
from torchvision import transforms # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader # 데이터를 모델에 사용할 수 있도록 적재해 주는 라이브러리
from torch.utils.data import random_split
from torch.utils.data import ConcatDataset
import numpy as np 
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchsummary import summary

- training 데이터에서 NORMAL 이미지 수가 PNEUMONIA 이미지 수보다 1/4정도 적음 (1341/3875)
- training의 NORMAL데이터를 두배로 불려주는 코드
- [참조](https://towardsdatascience.com/fastai-bag-of-tricks-experiments-with-a-kaggle-dataset-part-1-135e46da72f2)


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import glob


class additional_normal(Dataset):
    def __init__(self, root, transform):
        self.filepaths = glob.glob(root+'/train/NORMAL/*.jpeg')
        self.labels = 0
        self.transform = transform
        
    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, index):
        # Pytorch dataset에서는 image데이터를 PIL 형태로 읽음
        img_path = self.filepaths[index]
        img = Image.open(img_path).convert('RGB')
        img_transformed = self.transform(img)
        label = 0
        return img_transformed, label

In [ ]:
data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray'

import albumentations as A
from albumentations.pytorch import ToTensorV2
train_transform =transforms.Compose([transforms.Resize([224, 224]),
                               transforms.RandomHorizontalFlip(p=0.4),
                               transforms.RandomAdjustSharpness(sharpness_factor=2),
                               transforms.RandomAutocontrast(p=1),
                               transforms.RandomRotation(degrees=(-30, 30)),
                               transforms.RandomCrop(size=(224, 224)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))
                                ])
test_transform = transforms.Compose([transforms.Resize([224, 224]),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))])

trainset = datasets.ImageFolder(root= data_dir + "/train/", transform=train_transform)
additional_trainset_normal = additional_normal(root=data_dir, transform=train_transform)
trainset = ConcatDataset([trainset, additional_trainset_normal])
validset = datasets.ImageFolder(root= data_dir + "/val/", transform=test_transform)
testset = datasets.ImageFolder(root= data_dir + "/test/", transform=test_transform)

In [ ]:
len(trainset)
# len(testset)

# 2. 데이터 시각화

In [ ]:
labels_map = {0:'NORMAL', 1:'PNEUMONIA'} #정상상태와 폐렴상태를 구분하는 딕셔너리 작성 

fig = plt.figure(figsize=(16, 16)) # 차트의 기본크기 설정 단위 inch
columns = 5 # 5 열 
rows = 5 # 5 행
search_idx = 0 # 순회를 위한 탐색 인덱스 0으로 설정
for i in range(1,columns*rows +1): # 시작이 0이기에 1로 설정하고 행렬곱에 +1 순번까지 순회하여 총 갯수 확인
    while trainset[search_idx][1] != 0: # 훈련세트의 배열이 존재할 경우,
        search_idx+=1 #탐색 인덱스 값을 추가 
    img = trainset[search_idx][0].permute(1, 2, 0) #permute()는 파이토치에서 contiguous tensor(메모리가 인접한 텐서)의 형변환이다.
  ## img = img * 0.5 + 0.5 # unnormalize (normalize : (data - 0.5)/0.5))
    fig.add_subplot(rows, columns, i) # 행 수와 열 수 그리고 i값의 순회에 따라 서브플롯 추가, 따라서 5X5 플롯 생성
    plt.imshow(img) #이미지 위치마다 디렉토리가 지정된 이미지를 표시  
    ## 정답 :trainset[rnd_idx][1] - 정수
    plt.title(labels_map[trainset[search_idx][1]])  #순회하는 인덱스가 어떤 트레인셋에 포함되어있는지 확인하고,  normal 표시
    plt.axis("off") #축표시 제거 
    search_idx+=1 #형변환과 플롯표시가 끝났으므로 탐색인덱스 하나 추가. 
    

In [ ]:
fig = plt.figure(figsize=(16, 16)) #위와 상동하나, pneumonia를 표시하는 순서

search_idx = -1400 
for i in range(1,columns*rows +1):
    while trainset[search_idx][1] != 1:
        search_idx-=1
    img = trainset[search_idx][0].permute(1, 2, 0) 
  # img = img * 0.5 + 0.5 # unnormalize (normalize : (data - 0.5)/0.5))
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    # 정답 :trainset[rnd_idx][1] - 정수
    plt.title(labels_map[trainset[search_idx][1]])
    plt.axis("off")
    search_idx-=1

In [ ]:
print(len(trainset))
print(len(validset))
print(len(testset))

In [ ]:
batch_size = 16 # 중요한 하이퍼 파라미터, 16 이하로 사용하는것이 성능에 좋다고 알려져 있음
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True) # 훈련용
validloader = DataLoader(validset, batch_size=batch_size, shuffle=False) # 검증용
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False) # 테스트용

In [ ]:
from torch import nn
from torch import optim
import torch.nn.functional as F

import torchvision.models as models

In [ ]:
model = models.vgg16(pretrained=True)
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
# model.fc = nn.Linear(in_features=512, out_features=2, bias=True)

model.classifier[3] = nn.Linear(in_features=4096, out_features=512, bias=True)
model.classifier[6] = nn.Linear(in_features=512, out_features=2, bias=True)

for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

model.to(device)

In [ ]:
from torchsummary import summary
summary(model, (3, 244, 244))

In [ ]:
# !pip install torchmetrics
from torchmetrics import F1
from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()

In [ ]:
learning_rate = 0.0001
criterion = nn.CrossEntropyLoss() # 손실함수
f1 = F1(num_classes=2).to(device)
# criterion = F1(num_classes=3)
# optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9) # 옵티마이저 : Momentum
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001, betas=[0.9,0.999]) # 규제의 강도 : weight_decay=0.001
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=7, factor=0.1, verbose=True)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, verbose=True)

In [ ]:
def validation(model, testloader, criterion):
    f1_score = 0
    test_loss = 0

    with torch.no_grad(): # 예측에는 gradient가 필요 없음
        for images, labels in testloader: # 1 iteration마다 배치 단위로 image와 label을 가져옴
            images, labels = images.to(device), labels.to(device) # data를 GPU로 보내기
            # Not Flatten!!!
            #images.resize_(images.shape[0], 784) # batch size x 784
            
            logits = model.forward(images) # 입력 : batch_size, 28, 28
            
            loss = criterion(logits, labels)
            test_loss += loss.item()

            probs = F.softmax(logits, dim=1)
            _, preds = torch.max(probs, 1) # preds = probs.max(dim=1)[1]
            f1_score += f1(labels, preds).item()

    return test_loss, f1_score

In [ ]:
def train(model, epochs, criterion, optimizer):
    steps = 0
    # 1 에폭(epoch)당 반복수
    #iter_per_epoch = len(trainset) / batch_size  
    iter_per_epoch = len(trainloader) 
    print_every = iter_per_epoch
    best_f1 = 0
    train_loss_list = []
    val_loss_list = []
    f1_score_list = []
    e = []

    # for Early stopping
    min_loss = 1000
    trigger = 0
    patience = 10 # monitoring 하는 test loss가 더이상 줄지 않는 횟수를 몇번이나 참을지..

    for epoch in range(epochs):
      model.train()
      train_loss = 0
      for images, labels in tqdm(trainloader): #iterator에서 batch_size씩 가져와 images와 labels에 전달
        steps += 1
        images, labels = images.to(device), labels.to(device) # data를 GPU로 보내기
        # not Flatten!!
        # images.resize_(images.size()[0], 784)
        
        optimizer.zero_grad()

        # 1. Forward (Loss 얻게됨)
        output = model.forward(images)
        loss = criterion(output, labels)

        # 2. Backward (Gradient가 오차역전파로 구해짐)
        loss.backward()

        # 3. Gradient Descent (Model parameter update) :  W <- W-lr*Gradient
        optimizer.step() 

        train_loss += loss.item() # 1 epoch 당 누적된 로스
      
        if steps % print_every == 0 : 
            model.eval() # 배치 정규화, 드롭아웃 이 적용될 경우 model.forward 연산이 training 때와 다르므로 반드시 설정
            valid_loss, f1_score = validation(model, validloader, criterion)

#             writer.add_scalar("Loss/train", train_loss/len(trainloader), epoch)
#             writer.add_scalar("Loss/test", valid_loss/len(validloader), epoch)
#             writer.add_scalar("F1 Score",f1_score/len(validloader), epoch)
            

            print("Epoch : {}/{}...".format(epoch+1, epochs),
                  " Train Loss : {:.3f}".format(train_loss/len(trainloader) ),
                  " Valid Loss : {:.3f}".format(valid_loss/len(validloader)),
                  " F1 Score : {:.3f}".format(f1_score/len(validloader)))
        
            train_loss_list.append(train_loss/len(trainloader))
            val_loss_list.append(valid_loss/len(validloader))
            f1_score_list.append(f1_score/len(validloader))
            e.append(epoch)

            
            # Best model 저장
            if f1_score > best_f1:
                torch.save(model.state_dict(), "best_checkpoint.pth")
                best_f1 = f1_score

            # Early stopping Start 
            if valid_loss > min_loss: 
                trigger +=1
                print("trigger :", trigger)
                if trigger >= patience: 
                    print("Early Stopping!!!")
                    print("Train step is finished!!")
#                     writer.flush() 
                    return
            else: 
                min_loss = valid_loss
                trigger = 0
                # print("min loss set to current valid loss!!!")
            # Early stopping End

            train_loss = 0
            model.train()

            # Learning Rate Scheduler 
            scheduler.step(valid_loss)
    return train_loss_list, val_loss_list, f1_score_list, e

#     writer.flush()


In [ ]:
train_loss_ls, valid_loss_ls, f1_score_ls, e = train(model, 10, criterion, optimizer)

In [ ]:
plt.title("Train-Val Loss")
plt.plot(e,train_loss_ls,label="train")
plt.plot(e,valid_loss_ls,label="val")
plt.ylabel("Loss")
plt.xlabel("Training Epochs")
plt.legend()
plt.show()

# # plot accuracy progress
# plt.title("Train-Val Accuracy")
# plt.plot(range(1,num_epochs+1),metric_hist["train"],label="train")
# plt.plot(range(1,num_epochs+1),metric_hist["val"],label="val")
# plt.ylabel("Accuracy")
# plt.xlabel("Training Epochs")
# plt.legend()
# plt.show()

In [ ]:
def evaluate(model, testloader):
    f1_accuracy = 0
    test_loss = 0
    model.eval()
    with torch.no_grad(): # 예측에는 gradient가 필요 없음
        for images, labels in testloader: # 1 iteration마다 배치 단위로 image와 label을 가져옴
            images, labels = images.to(device), labels.to(device)
            # not Flatten!
            #images.resize_(images.shape[0], 784) # batch size x 784
            logits = model.forward(images) # 입력 : batch * height * width
            
            loss = criterion(logits, labels)
            test_loss += loss.item()

            probs = F.softmax(logits, dim=1)
            _, preds = torch.max(probs, 1) # preds = probs.max(dim=1)[1]
            correct = (preds == labels).sum()
            print(labels, preds)
            f1_score = f1(labels, preds)
#             accuracy = correct / images.shape[0]
            f1_accuracy += f1_score.item()

    print("Test Loss : ", test_loss/len(testloader))
    print("Average F1 : ", f1_accuracy/len(testloader))

In [ ]:
evaluate(model, testloader)